In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
import dill
import os
import pickle
import pandas as pd

from lib_EGNN_Pytorch import utils
from lib_EGNN_Pytorch.data_preprocessing import Pre_utils_graphsaint
from lib_EGNN_Pytorch.app.meta_clustergcn import base_exec_graphsage
from lib_EGNN_Pytorch.meta_learn import batch_machine

###  load data settings

In [2]:
datapath = '/home/xiangli/projects/tmpdata/GCN/GraphSaint/'

# read the total epoch number from the yml file to determine the mini_epoch_num and eval_train_every
data_name = 'flickr'
train_config_yml = './clustergcn_config/flickr2_e.yml'
multilabel_tag = False

# data_name = 'PPI_small'
# train_config_yml = './table2/ppi2_e.yml'

working_dir = os.path.join('/home/xiangli/projects/GCN_program/Workshop_local/Meta_Learn_GCN/',
                              "graphsage_res/",
                               data_name.lower()
                          )

prepare_data_folder = os.path.join(working_dir, 'prepare_data/')
img_path = os.path.join(working_dir, 'res_images/')

core_par_sampler = 1
samples_per_processor = -(-200 // core_par_sampler) # round up division
eval_train_every = 5  # period to record the train loss

### ================ Start to do flexible settings according to different dataset: 


tune_param_name = 'mini_epoch_num'
tune_val_label_list = [1, 5] 
tune_val_list = [val for val in tune_val_label_list]

snapshot_period = 2   # period when to take a snapshot of the model for validation later

# refer to the yml file to decide the training period:
model_epoch_list = list(range(snapshot_period, 16, snapshot_period))    # snapshot epoch list for validation

trainer_list = list(range(1))

tune_val_label = tune_val_label_list[0]
tune_val = tune_val_list[0]
trainer_id = trainer_list[0]

### prepare data

In [4]:
# =============== Step1 *** prepare for the batches, models, model_evaluation
train_params, train_phases, train_data, arch_gcn = Pre_utils_graphsaint.train_setting(data_name.lower(), datapath, train_config_yml)

base_exec_graphsage.prepare(working_dir, train_data, train_params, arch_gcn)
train_phase_file_name = os.path.join(prepare_data_folder, 'model_train_phase.pkl')

utils.save_info_use_dill(train_phase_file_name, train_phases, exist_ok = True)


/home/xiangli/projects/GCN_program/meta_learn_GCN/PC_src_code/all_in_one/lib_meta_learn_Pytorch/batch_machine.py:413: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  train_config = yaml.load(f_train_config)


Loading training data..
Done loading training data..


### Training process

In [5]:
# ============== Step2 *** conduct the training process
train_input_file_name = os.path.join(prepare_data_folder, 'model_train_input.pkl') 
with open(train_input_file_name, "rb") as fp:
    minibatch, model = dill.load(fp)

train_phase_file_name = os.path.join(prepare_data_folder, 'model_train_phase.pkl')
with open(train_phase_file_name, "rb") as fp:
    train_phases = dill.load(fp)


### ==========================  single run of training
tune_model_folder = os.path.join(working_dir, 'model_snapshot', f'tune_{tune_param_name}_{tune_val_label}',
                        f'model_trainer_{trainer_id}/')
    
os.makedirs(tune_model_folder, exist_ok=True)

# to apply any tuning values
total_time_train, time_upload = base_exec_graphsage.train_investigate(tune_model_folder, train_phases, model, minibatch, eval_train_every, 
                                snapshot_every = snapshot_period, mini_epoch_num = 5, multilabel = multilabel_tag, 
                          core_par_sampler = core_par_sampler, samples_per_processor = samples_per_processor)    



# for tune_val_label, tune_val in zip(tune_val_label_list, tune_val_list):
#     for trainer_id in trainer_list:
#         execute_train_investigate(img_path, working_dir, train_phases, model, minibatch, eval_train_every, 
#                                   tune_param_name, tune_val_label, tune_val, trainer_id = trainer_id,
#                                   snapshot_every = snapshot_period, mini_epoch_num = 5, multilabel = multilabel_tag, input_neigh_deg = [10, 5],
#                                   core_par_sampler = core_par_sampler, samples_per_processor = samples_per_processor)


START PHASE    0


/home/xiangli/projects/GCN_program/meta_learn_GCN/PC_src_code/all_in_one/lib_meta_learn_Pytorch/samplers.py:61: RuntimeWarning: divide by zero encountered in true_divide
  self.adj_train_norm = scipy.sparse.dia_matrix((1 / self.deg_train,0),shape=adj_train.shape).dot(adj_train)


sampling 200 subgraphs:   time = 0.867 sec
Epoch    1, Batch ID 0
Epoch    2, Batch ID 0
Epoch    3, Batch ID 0
Epoch    4, Batch ID 0
Epoch    5, Batch ID 0
Epoch    1, Batch ID 1
Epoch    2, Batch ID 1
Epoch    3, Batch ID 1
Epoch    4, Batch ID 1
Epoch    5, Batch ID 1
Epoch    1, Batch ID 2
Epoch    2, Batch ID 2
Epoch    3, Batch ID 2
Epoch    4, Batch ID 2
Epoch    5, Batch ID 2
Epoch    1, Batch ID 3
Epoch    2, Batch ID 3
Epoch    3, Batch ID 3
Epoch    4, Batch ID 3
Epoch    5, Batch ID 3
Epoch    6, Batch ID 0
Epoch    7, Batch ID 0
Epoch    8, Batch ID 0
Epoch    9, Batch ID 0
Epoch   10, Batch ID 0
Epoch    6, Batch ID 1
Epoch    7, Batch ID 1
Epoch    8, Batch ID 1
Epoch    9, Batch ID 1
Epoch   10, Batch ID 1
Epoch    6, Batch ID 2
Epoch    7, Batch ID 2
Epoch    8, Batch ID 2
Epoch    9, Batch ID 2
Epoch   10, Batch ID 2
Epoch    6, Batch ID 3
Epoch    7, Batch ID 3
Epoch    8, Batch ID 3
Epoch    9, Batch ID 3
Epoch   10, Batch ID 3
Epoch   11, Batch ID 0
Epoch   12, Ba

### Investigate validation

In [6]:
# ================ Step3*** investigate validation:
evaluation_input_file_name = os.path.join(prepare_data_folder, 'model_eval_input.pkl')
with open(evaluation_input_file_name, "rb") as fp:
    minibatch_eval, model_eval = dill.load(fp)


# perform the evaluation
tune_model_folder = os.path.join(working_dir, 'model_snapshot', f'tune_{tune_param_name}_{tune_val_label}',
                                    f'model_trainer_{trainer_id}/')

validation_res_folder = os.path.join(img_path, 'validation_res', f'tune_{tune_param_name}_{tune_val_label}',
                                    f'validation_trainer_{trainer_id}/')
os.makedirs(validation_res_folder, exist_ok=True)

# start evaluation:
for validation_epoch in model_epoch_list:
    res = batch_machine.evaluate(tune_model_folder, minibatch_eval, model_eval, validation_epoch)    
    validation_res_file_name = os.path.join(validation_res_folder, f'model_epoch_{validation_epoch}')
    with open(validation_res_file_name, "wb") as fp:
        pickle.dump(res, fp)
    
# for tune_val_label, tune_val in zip(tune_val_label_list, tune_val_list):
#     for trainer_id in trainer_list:
#         execute_validation_investigate(img_path, working_dir, minibatch_eval, model_eval, model_epoch_list, 
#                                 tune_param_name, tune_val_label, tune_val, trainer_id = trainer_id)



during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
during the evaluation step, report the input matrices size: 
adj_subgraph si

### Investigate test

In [7]:
# ================= Step4*** investigate test:
evaluation_input_file_name = os.path.join(prepare_data_folder, 'model_eval_input.pkl')
with open(evaluation_input_file_name, "rb") as fp:
    minibatch_eval, model_eval = dill.load(fp)
    

### ======================== Perform the single run of the test =======================
f1mic_best, ep_best = 0, -1
validation_res_folder = os.path.join(img_path, 'validation_res', f'tune_{tune_param_name}_{tune_val_label}',
                                f'validation_trainer_{trainer_id}/')


for validation_epoch in model_epoch_list:
    validation_res_file_name = os.path.join(validation_res_folder, f'model_epoch_{validation_epoch}')
    with open(validation_res_file_name, "rb") as fp:
        f1mic_val, f1mac_val = pickle.load(fp)

    if f1mic_val > f1mic_best:
        f1mic_best, ep_best = f1mic_val, validation_epoch

# use the selected model to perform on the test
tune_model_folder = os.path.join(working_dir, 'model_snapshot', f'tune_{tune_param_name}_{tune_val_label}',
                                f'model_trainer_{trainer_id}/')

# return 1) micro-f1 ;  2) macro-f1
res = batch_machine.evaluate(tune_model_folder, minibatch_eval, model_eval, ep_best, mode = 'test')        


# for tune_val_label, tune_val in zip(tune_val_label_list, tune_val_list):
#     for trainer_id in trainer_list:
#         execute_test_tuning(img_path, working_dir, minibatch_eval, model_eval, model_epoch_list, 
#                                 tune_param_name, tune_val_label, tune_val, trainer_id = trainer_id)

during the evaluation step, report the input matrices size: 
adj_subgraph size :  torch.Size([89250, 89250]); 	 feat_subg size : torch.Size([89250, 500])
